In [811]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import pandas
from sklearn.metrics import mean_squared_error

In [812]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [813]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [814]:
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d

In [815]:
answers = {}

In [816]:
def UsersGames(data):
    usergames = defaultdict(set)
    gameusers = defaultdict(set)
    for u, g, d in data:
        usergames[u].add(g)
        gameusers[g].add(u)
    return usergames, gameusers

In [817]:
allHours = []
for l in readJSON("train.json.gz"):
    allHours.append(l)

In [818]:
hoursTrain = allHours[:165000]
hoursValid = allHours[165000:]

In [674]:
##################################################
# Play prediction                                #
##################################################

In [746]:
gameCount = defaultdict(int)
totalPlayed = 0

for user,game,_ in hoursTrain:
    gameCount[game] += 1
    totalPlayed += 1


# gameCount = defaultdict(int)
# gameHours = defaultdict(float)

# for user, game, d in hoursTrain:
#     gameCount[game] += 1
#     gameHours[game] += d['hours']
#     totalPlayed += 1




# additional_factor_weight =0.5 

# mostPopular = [(gameCount[x] * (1 - additional_factor_weight) + 
#                 (gameHours[x] / gameCount[x]) * additional_factor_weight, x) 
#                for x in gameCount]
mostPopular = [(gameCount[x], x) for x in gameCount]

mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPlayed/2: break

def get_prediction(u,g):
    if g in return1:
        return 1
    else:
        return 0

In [747]:
### Question 1

In [748]:
# Evaluate baseline strategy
allGames = set([g for u, g, d in hoursTrain + hoursValid])

usergames,gameusers = UsersGames(hoursTrain + hoursValid)

augmentedValidationSet = []

valid1 = []
valid2 = []

for u,i,d in hoursValid:
    valid1.append((u, i, 1))
    valid2.append((u, random.choice(list(allGames - usergames[u])), 0))
augmentedValidationSet = valid1 + valid2

In [749]:
predictions = [1 if get_prediction(u, g)==p else 0 for u,g,p in augmentedValidationSet]

accuracy = sum(predictions)/len(augmentedValidationSet)

In [750]:
accuracy

0.6824682468246824

In [751]:
answers['Q1'] = accuracy

In [752]:
assertFloat(answers['Q1'])

In [753]:
### Question 2

In [754]:
# Improved strategy
threshold = 2.7 * totalPlayed / 5

return12 = set()
count2 = 0
for ic, i in mostPopular:
  count2 += ic
  return12.add(i)
  if count2 > threshold: break

def get_prediction2(u,g):
    if g in return12:
        return 1
    else:
        return 0

In [755]:
# Evaluate baseline strategy
predictions = [1 if get_prediction2(u, g)==p else 0 for u,g,p in augmentedValidationSet]

accuracy = sum(predictions)/len(augmentedValidationSet)

In [756]:
accuracy

0.6887688768876887

In [757]:
answers['Q2'] = [accuracy, 100*2/3]

In [758]:
assertFloatList(answers['Q2'], 2)

In [759]:
### Question 3/4

In [760]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom


In [761]:
usergames_train,gameusers_train = UsersGames(hoursTrain)

In [762]:
def predictJaccard(u,g):
    maxJaccrd = 0
    threshold34 = 0.03035
    predictionsJaccard = []
    gamesset = usergames_train.get(u)
    if gamesset is None:
        return 0
    for g1 in gamesset:
        if g==g1:
            continue
        j = Jaccard(gameusers_train.get(g1),gameusers_train.get(g))
        if j>maxJaccrd:
            maxJaccrd = j
    if maxJaccrd > threshold34:
        return 1
    else:
        return 0

In [763]:
predictions = [predictJaccard(u, g)==p for u,g,p in augmentedValidationSet]

accuracy3 = sum(predictions)/len(augmentedValidationSet)

In [764]:
accuracy3

0.6753675367536753

In [765]:
threshold = 2 * totalPlayed / 3

return13 = set()
count3 = 0
for ic, i in mostPopular:
  count3 += ic
  return13.add(i)
  if count3 > threshold: break

def get_prediction3(u,g):
    if g in return13:
        return 1
    else:
        return 0

In [766]:
predictions = [1 if get_prediction3(u, g) or predictJaccard(u, g) else 0 for u,g,p in augmentedValidationSet]
accuracy4 = sum([p == l for p, l in zip([p for u,d,p in augmentedValidationSet], predictions)]) / len(augmentedValidationSet)
accuracy4

0.6836183618361836

In [656]:
answers['Q3'] = accuracy3
answers['Q4'] = accuracy4

In [657]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [664]:
return14 = set()
count4 = 0
for ic, i in mostPopular:
  count4 += ic
  return14.add(i)
  if count4 > threshold: break

def get_prediction4(u,g):
    if g in return14:
        return 1
    else:
        return 0

In [665]:
predictions = open("predictions_Played.csv", 'w')
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    baselinepre = get_prediction4(u, g)
    jpre = predictJaccard(u, g)
    if baselinepre or jpre:
        pred = 1
    else:
        pred = 0

    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()

In [598]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [162]:
##################################################
# Hours played prediction                        #
##################################################

In [936]:
trainHours = [r[2]['hours_transformed'] for r in hoursTrain]
globalAverage = sum(trainHours) * 1.0 / len(trainHours)

In [937]:
### Question 6

In [938]:
hoursPerUser = {}
hoursPerItem = {}
for u, g, d in hoursTrain:
    t = d.get('hours_transformed')
    if u not in hoursPerUser:
        hoursPerUser[u] = set()
    hoursPerUser[u].add(t)
    if g not in hoursPerItem:
        hoursPerItem[g] = set()
    hoursPerItem[g].add(t)

In [939]:
betaU = {}
betaI = {}
for u in hoursPerUser:
    betaU[u] = 0

for g in hoursPerItem:
    betaI[g] = 0

In [940]:
alpha = globalAverage # Could initialize anywhere, this is a guess
alpha

3.716088074007024

In [941]:
lambda_ = 1
iter = 100
tolerance = 1e-5 
pmse = 0
converged = False

In [942]:
usergameHours = {}
for u, g, d in hoursTrain:
    usergameHours[(u,g)] = d['hours_transformed']

In [1002]:
def iterate(lamb):
    alpha = globalAverage
    betau_iterate = {}
    betai_iterate = {}
    for u in hoursPerUser:
        betau_iterate[u] = 0
    
    for g in hoursPerItem:
        betai_iterate[g] = 0
    iter = 100
    tolerance = 1e-8
    pmse = float("inf")
    converged = False
    tolerance_count=3
    
    for iteration in range(iter):
        alpha = sum([ usergameHours[d] - betau_iterate[d[0]] - betai_iterate[d[1]] for d in usergameHours])/len(usergameHours)
        
        for u in usergames_train:
            betau_iterate[u] = sum([ usergameHours[(u,g)] - alpha - betai_iterate[g] for g in usergames_train[u]])/(lamb + len(usergames_train[u]))
    
        for g in gameusers_train:
            betai_iterate[g] = sum([ usergameHours[(u,g)] - alpha - betau_iterate[u] for u in gameusers_train[g]])/(lamb + len(gameusers_train[g]))
    
        predictions = [alpha + betau_iterate[u] + betai_iterate[g] for u,g,d in hoursValid]
        y = [d['hours_transformed'] for u,g,d in hoursValid]
    
        mse = mean_squared_error(predictions, y)
        
        if mse > pmse:
            break
        else:
            pmse = mse

    return alpha, betau_iterate, betai_iterate, mse

In [975]:
answers['Q6'] = mse1

In [946]:
assertFloat(answers['Q6'])

In [876]:
### Question 7

In [790]:
betaUs = [(betaU[u], u) for u in betaU]
betaIs = [(betaI[i], i) for i in betaI]
betaUs.sort()
betaIs.sort()

print("Maximum betaU = " + str(betaUs[-1][1]) + ' (' + str(betaUs[-1][0]) + ')')
print("Maximum betaI = " + str(betaIs[-1][1]) + ' (' + str(betaIs[-1][0]) + ')')
print("Minimum betaU = " + str(betaUs[0][1]) + ' (' + str(betaUs[0][0]) + ')')
print("Minimum betaI = " + str(betaIs[0][1]) + ' (' + str(betaIs[0][0]) + ')')

Maximum betaU = u60898505 (5.404040043113816)
Maximum betaI = g17604638 (5.079399440514896)
Minimum betaU = u13037838 (-2.974394854201803)
Minimum betaI = g84397720 (-3.123343276752127)


In [791]:
answers['Q7'] = [betaUs[-1][0], betaUs[0][0], betaIs[-1][0], betaIs[0][0]]

In [792]:
answers['Q7']

[5.404040043113816, -2.974394854201803, 5.079399440514896, -3.123343276752127]

In [793]:
assertFloatList(answers['Q7'], 4)

In [794]:
### Question 8

In [1003]:
# Better lambda...
bestmse=float("inf")
bestlamb = 0
for l in range(480,500):
    _,_,_,t = iterate(l/100)
    print(t)
    if t < bestmse:
        bestmse = t
        bestlamb = l/100
bestlamb

2.9906434378428144
2.990641345398677
2.990639404088878
2.9906376135300303
2.990635973339882
2.990634483137326
2.9906331425423853
2.9906319511762174
2.9906309086611036
2.9906300146204496
2.990629268678779
2.9906286704617333
2.9906282195960614
2.9906279157096223
2.990627758431377
2.9906277473913887
2.9906278822208137
2.990628162551902
2.9906285880179917
2.990629158253506


4.95

In [1004]:
bestmse

2.9906277473913887

In [1005]:
alpha, betaU, betaI, mse5 = iterate(bestlamb)
mse5

2.9906277473913887

In [841]:
answers['Q8'] = (l, bestmse)

In [842]:
assertFloatList(answers['Q8'], 2)

In [994]:
predictions = open("predictions_Hours.csv", 'w')
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    bu = betaU[u]
    bi = betaI[g]
    
    _ = predictions.write(u + ',' + g + ',' + str(alpha + bu + bi) + '\n')

predictions.close()

In [935]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()